<a href="https://colab.research.google.com/github/HoYoungChun/financial_data_analysis/blob/main/crawling2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## campany.csv 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd

csv_url = '/content/drive/MyDrive/Colab/financial_data_analysis/company.csv'

company = pd.read_csv(csv_url, encoding='utf-8')

## BenjaminGraham

#### 레벨: 초급
#### 스타일: 밸류+퀄리티
#### 기대CAGR: 15%
#### 종목개수: 20~30개
#### 매수전략:PER 10 이하(종목이 충분할 경우 PER 5 이하로 제한), 부채비율 50% 이하


### Basic Setting
### 에프앤가이드

In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
print(SNAP_URL)

https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701


In [ ]:
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'
print(RATIO_URL)

https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701


In [ ]:
# 삼성전자 종목코드
sample_code = '005930'

In [ ]:
print(SNAP_URL.format(sample_code))

https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701


### GET PER data

In [ ]:
snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content
snap_tree = html.fromstring(snap_content)

In [ ]:
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[2]/dd')[0].text

In [ ]:
per = float(per)
per

13.0

In [ ]:
per<10

False

### GET Debt Ratio data

In [ ]:
# 부채비율 크롤링
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)

In [ ]:
dept_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text

In [ ]:
dept_ratio = float(dept_ratio)
dept_ratio

43.2

In [ ]:
dept_ratio<50

True

### GET All stock code

In [ ]:
print(company.shape)
company.head(10)

(2022, 11)


,분류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,구글코드
0,코스피,두산우,000155,회사 본부 및 경영 컨설팅 서비스업,"청주,소주,인쇄회로기판(인쇄회로용동박적층판),관련전자부품,기계 제조,판매/출판사업,...",1973-06-29,12월,"박정원, 이재경",http://www.doosancorp.co.kr,서울특별시,KRX:000155
1,코스피,한화3우B,00088K,기타 화학제품 제조업,"화약,화공품,자동선반류,공작기계 제조,전자교환기,건설,정보통신,종합무역/임직원 및 ...",1976-06-24,12월,"최양수, 이태종, 김연철, 이민석",http://www.hanwhacorp.co.kr,서울특별시,KRX:00088K
2,코스피,S-Oil우,010955,석유 정제품 제조업,"석유제품,가스,윤활기유,윤활유,그리스,석유화학제품 제조,도매",1987-05-27,12월,Othman Al-Ghamdi,http://www.s-oil.com,서울특별시,KRX:010955
3,코스피,LG화학우,051915,기초 화학물질 제조업,"유화/기능/합성수지,재생섬유소,산업재,리튬이온전지,평광판,PVC 제조,도매",2001-04-25,12월,박진수,http://www.lgchem.com,서울특별시,KRX:051915
4,코스피,한국금융지주우,071055,기타 금융업,금융지주회사,2003-07-21,12월,김남구,http://www.koreaholdings.com,서울특별시,KRX:071055
5,코스피,GS우,078935,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허창수, 정택근 (각자 대표이사)",NaN,서울특별시,KRX:078935
6,코스피,SK이노베이션우,096775,기타 금융업,"석유정제,석유화학제품,윤활유제품,아스팔트제품,의약중간체 제조,판매,유전개발,대체에너지사업",2007-07-25,12월,김준,http://www.SKinnovation.com,서울특별시,KRX:096775
7,코스피,코오롱인더우,120115,화학섬유 제조업,"산업자재, 화학, 필름",2010-02-01,12월,"이웅열, 박동문, 안태환",NaN,경기도,KRX:120115
8,코스피,삼성화재우,000815,보험업,"손해보험의 원수,재보험,운용자산의 투자활동",1975-06-30,12월,안민수,http://www.samsungfire.com,서울특별시,KRX:000815
9,코스피,현대차2우B,005387,자동차용 엔진 및 자동차 제조업,"자동차(승용차,버스,트럭,특장차),자동차부품,자동차전착도료 제조,차량정비사업",1974-06-28,12월,"정몽구, 윤갑한, 이원희(각자 대표이사)",http://www.hyundai.com,서울특별시,KRX:005387


In [ ]:
code_list = company['종목코드']

In [ ]:
sample_code_list = code_list[1000:1050]
sample_code_list

1000    207930
1001    214270
1002    189690
1003    213420
1004    206660
1005    060480
1006    206640
1007    208640
1008    193250
1009    204630
1010    200470
1011    200670
1012    208710
1013    189860
1014    067390
1015    080580
1016    208350
1017    160550
1018    124500
1019    142280
1020    187220
1021    208370
1022    207720
1023    208140
1024    149980
1025    084650
1026    200710
1027    200780
1028    207760
1029    196170
1030    206400
1031    178920
1032    140520
1033    173940
1034    205470
1035    200230
1036    194510
1037    204840
1038    196490
1039    204620
1040    205500
1041    205100
1042    191420
1043    192440
1044    182690
1045    143540
1046    203650
1047    041920
1048    194480
1049    203690
Name: 종목코드, dtype: object

In [ ]:
def crawler(li):
  SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
  RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

  result = {}
  for code in tqdm(li):
    try:
      #get pbr
      snap_url = SNAP_URL.format(code)
      snap_content = requests.get(snap_url).content
      snap_tree = html.fromstring(snap_content)

      pbr = snap_tree.xpath('//*[@id="corp_group2"]/dl[4]/dd')[0].text
      pbr = float(pbr)

      #get debt ratio, roa
      ratio_url = RATIO_URL.format(code)
      ratio_content = requests.get(ratio_url).content
      ratio_tree = html.fromstring(ratio_content)

      dept_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
      dept_ratio = float(dept_ratio)

      roa = ratio_tree.xpath('//*[@id="p_grid1_17"]/td[5]')[0].text
      roa = float(roa)

      #company name
      name = company[company['종목코드'] == code]['회사명'].values[0]
      result[name]=[code, roa, dept_ratio, pbr]

    except(TypeError, AttributeError, IndexError, ValueError) as err: #alias
      pass

  result_df = pd.DataFrame(result)
  result_df = result_df.transpose()
  result_df.columns=['Code','ROA','Debt_Ratio', 'PBR']
  return result_df

In [ ]:
result_df = crawler(sample_code_list)
result_df

100%|██████████| 50/50 [01:21<00:00,  1.64s/it]


,Code,ROA,Debt_Ratio,PBR
포시에스,189690,5.7,3.1,1.39
덕산네오룩스,213420,21.6,13.8,7.11
국일신동,060480,3.3,11,1.25
바디텍메드,206640,29.5,26.5,4.26
썸에이지,208640,127,6.7,17.13
휴메딕스,200670,8.9,33.5,2.56
오킨스전자,080580,0.9,96.5,11.57
NEW,160550,0.8,149.5,2.63
녹십자엠에스,142280,5.4,111.5,4.13
정다운,208140,6.1,113.9,1.92


In [ ]:
!pip install tqdm

In [ ]:
copy_df = result_df.copy()

### Searching Stocks

In [ ]:
filtered_df = copy_df[(copy_df['ROA'] >= 5) & (copy_df['Debt_Ratio'] <= 50)& (copy_df['PBR'] > 0.2)]

In [ ]:
filtered_df.sort_values(by='PBR', ascending=True).head(30)

,Code,ROA,Debt_Ratio,PBR
슈피겐코리아,192440,16.4,18,1.02
포시에스,189690,5.7,3.1,1.39
메디아나,041920,12.7,15.7,1.78
휴메딕스,200670,8.9,33.5,2.56
엔터메이트,206400,22.2,31.7,2.97
랩지노믹스,084650,44.9,36.5,3.49
바디텍메드,206640,29.5,26.5,4.26
미스터블루,207760,19.3,33.5,5.07
SKC코오롱PI,178920,15.8,49.7,5.66
덕산네오룩스,213420,21.6,13.8,7.11
